# Comparing different solver 
(**Rectangular integration** )

### Importing the Model

In [1]:
using JuMP , Ipopt
import HSL_jll
using DataFrames, BenchmarkTools
include("../Problems/JuMP/space_Shuttle_JMP.jl")
include("../Benchmark/utils.jl")
results = DataFrame( :method => String[], 
                    :total_time => String[],
                    :objective_value => Float64[],
                    :iterations => Int64[]);


## Comparing Linear Solvers : HSL_MA57, HSL_MA27 and MUMPS

#### 1. MUMPS

In [2]:
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_optimizer_attribute(model,"linear_solver", "mumps")
b = @benchmark optimize!(model) evals=1 samples=1
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)
push!(results,["Ipopt + MUMPS",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Final latitude θ = 34.1790697°


#### 2. HSL_MA57

In [3]:
import HSL_jll
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma57")
b = @benchmark optimize!(model) evals=1 samples=1
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)
push!(results,["Ipopt + HSL_MA57",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

Final latitude θ = 34.1790697°


#### 3. HSL_MA27

In [4]:
import HSL_jll
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma27")
b = @benchmark optimize!(model) evals=1 samples=1
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)
push!(results,["Ipopt + HSL_MA27",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

Final latitude θ = 34.1790697°


>>>#### These 3 linear solvers give the same results for the rectangular integration problem.


## Comparing Nonlinear Solvers : Ipopt, MadNLP and Knitro

#### 1. Ipopt

> The reference solver is Ipopt with HSL_MA57 linear solver.

In [ ]:
import HSL_jll
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma57")
optimize!(model)  # Solve for the control and state
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)

#### 2. MadNLP

In [5]:
using  MadNLP
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,MadNLP.Optimizer) #default linear solver : Umfpack
b = @benchmark optimize!(model) evals=1 samples=1
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)
push!(results,["MadNLP + Umfpack",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);


Final latitude θ = 34.166426°


> the problem for not finding the same solutions with Ipopt + HSL_MA57 and MadNLP comes from the linear solvers used . MadNLP uses Umfpack by default, while we use HSL_MA57 with Ipopt.

In [10]:
using  MadNLP
using MadNLPHSL
model = space_Shuttle_JMP();
set_silent(model)
set_optimizer(model,()->MadNLP.Optimizer(linear_solver=Ma57Solver))
b = @benchmark optimize!(model) evals=1 samples=1
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)
push!(results,[ "MadNLP + HSL_MA57",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

Final latitude θ = 34.1790697°


# results

In [ ]:
results

In [56]:
using PrettyTables
using Colors
# Define the custom display function
hl_v = Highlighter(
           (results, i, j) -> (j == 3) && (results[i, j] != 34.1790697),
           crayon"red"
       );
header = ["Method", "Total Time", "Objective Value", "Iterations"];
# Apply the custom display function to the :objective_value column
pretty_table(
    results;
    formatters    = ft_printf("%.7f", 2:3),
    header        = header,
    header_crayon = crayon"yellow bold",
    highlighters  = (hl_v),
    tf            = tf_unicode_rounded
)

╭───────────────────┬──────────────┬─────────────────┬────────────╮
│            Method │   Total Time │ Objective Value │ Iterations │
├───────────────────┼──────────────┼─────────────────┼────────────┤
│     Ipopt + MUMPS │  2.1744118 s │      34.1790697 │         97 │
│  Ipopt + HSL_MA57 │  1.8534218 s │      34.1790697 │         97 │
│  Ipopt + HSL_MA27 │  2.2354654 s │      34.1790697 │         97 │
│  MadNLP + Umfpack │ 51.7091433 s │      34.1664260 │       1013 │
│ MadNLP + HSL_MA57 │ 70.3543391 s │      34.1790697 │        618 │
╰───────────────────┴──────────────┴─────────────────┴────────────╯
